# Mount

In [1]:
import os
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import shutil
import csv

from google.colab import drive
drive.mount('/content/drive')

# Set the path of the directory 'dataset'
dataset_dir = '/content/drive/MyDrive/SNU/4-1/4-1_인간컴퓨터상호작용/프로젝트/HCI_Project_Meal_Consultant/dataset'

Mounted at /content/drive


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

# fruit-360

In [18]:
dataset = 'fruits-360'

labels = ['Apple', 'Banana', 'Chestnut', 'Corn', 'Cucumber', 'Dates', 'Eggplant', 'Grape', 'Kaki',
          'Mandarine', 'Onion', 'Orange', 'Pear', 'Pepper', 'Potato', 'PotSweet', 'Tomato', 'Walnut']

dir_list = os.listdir(os.path.join(dataset_dir, dataset, 'Test'))
label_files_dict = {}

for label in labels:
    label_dir_list = [dirname for dirname in dir_list if label in dirname]
    
    label_files = []
    for label_dir in label_dir_list:
        label_dir_files = os.listdir(os.path.join(dataset_dir, dataset, 'Training', label_dir))
        label_files.extend([os.path.join(dataset_dir, dataset, 'Training', label_dir, filename) for filename in label_dir_files])
        label_dir_files = os.listdir(os.path.join(dataset_dir, dataset, 'Test', label_dir))
        label_files.extend([os.path.join(dataset_dir, dataset, 'Test', label_dir, filename) for filename in label_dir_files])
        
    label_files_dict[label] = label_files


num_samples = 3000
num_sub = 3
img_size = 100

storage_bucket = dataset + '_samples'
# samples_dir = os.path.join('drive', 'MyDrive', 'samples')
samples_dir = os.path.join('drive', 'MyDrive', 'samples2')

# check existence of the directory for generated sample files
try:
    os.listdir(samples_dir)
except:
    os.makedirs(samples_dir)

detection_boxes = []
label_count = {}
for label in labels:
    label_count[label] = 0

for n in range(num_samples):
    sample_labels = np.random.choice(labels, size=num_sub)
    for sample_label in sample_labels:
        label_count[sample_label] += 1
    sub_sample_files = [np.random.choice(label_files_dict[sample_label]) for sample_label in sample_labels]
    sub_sample_imgs = [cv2.imread(sub_sample_file, cv2.IMREAD_COLOR) for sub_sample_file in sub_sample_files]
    sub_sample_coords = np.random.randint(low=0, high=img_size, size=(num_sub, 2))
    sample = np.ones(shape=(2*img_size, 2*img_size, 3), dtype=np.uint8) * 255   # white empty img
    
    sample_filename = f'{n}.jpg'
    for i in range(num_sub):
        x, y = sub_sample_coords[i]
        sample[y:y+img_size, x:x+img_size] = np.minimum(sample[y:y+img_size, x:x+img_size], sub_sample_imgs[i])
        detection_boxes.append(['UNASSIGNED',
                                'gs://' + os.path.join(storage_bucket, 'samples2', sample_filename),
                                sample_labels[i],
                                x/(2*img_size), y/(2*img_size),
                                (x+img_size)/(2*img_size), y/(2*img_size),
                                (x+img_size)/(2*img_size), (y+img_size)/(2*img_size),
                                x/(2*img_size), (y+img_size)/(2*img_size)])
            
    cv2.imwrite(os.path.join(samples_dir, sample_filename), sample)

    if (n+1) % 100 == 0:
        print(f'{n+1} samples generated')

print(label_count)

with open(os.path.join(samples_dir, 'detection_boxes.csv'), 'w') as f:
    writer = csv.writer(f)
    writer.writerows(detection_boxes)

100 samples generated
200 samples generated
300 samples generated
400 samples generated
500 samples generated
600 samples generated
700 samples generated
800 samples generated
900 samples generated
1000 samples generated
1100 samples generated
1200 samples generated
1300 samples generated
1400 samples generated
1500 samples generated
1600 samples generated
1700 samples generated
1800 samples generated
1900 samples generated
2000 samples generated
2100 samples generated
2200 samples generated
2300 samples generated
2400 samples generated
2500 samples generated
2600 samples generated
2700 samples generated
2800 samples generated
2900 samples generated
3000 samples generated
{'Apple': 503, 'Banana': 512, 'Chestnut': 479, 'Corn': 478, 'Cucumber': 460, 'Dates': 562, 'Eggplant': 516, 'Grape': 528, 'Kaki': 516, 'Mandarine': 492, 'Onion': 518, 'Orange': 479, 'Pear': 477, 'Pepper': 490, 'Potato': 511, 'PotSweet': 506, 'Tomato': 479, 'Walnut': 494}
